This notebook is just a demonstration on how the SOAP vectors are obtained starting from the .xyz files. 
The soap.npz files were already generated and can be downloaded, so there is no real need to run this notebook. 
If you decide to run it anyways using production data, be prepared to wait for a while, depending on the power of your RAM. The output files will be called 'generated_soap*.npz'. 

In [ ]:
from data import DATA_3DCD, DATA_MP

In [ ]:
%run ./modules.ipynb

In [ ]:
if DATA_3DCD.soap.exists() and DATA_MP.soap.exists():
    print("Data alreeady present, no need to execute this notebook.")

In [ ]:
frames_3dcd = ase.io.read(DATA_3DCD.structures, index=":")
frames_mp = ase.io.read(DATA_MP.structures, index=":")
len(frames_3dcd), len(frames_mp)

In [ ]:
species = []
for frame in tqdm(frames_3dcd):
    for n in list(set(frame.numbers)):
        if n not in species:
            species.append(n)
for frame in tqdm(frames_mp):
    for n in list(set(frame.numbers)):
        if n not in species:
            species.append(n)
len(species)

In [ ]:
hypers = {
    "cutoff": 3.5,
    "max_angular": 4,
    "max_radial": 4,
    "atomic_gaussian_width": 0.5,
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "radial_basis": {"SplinedGto": {"accuracy": 1e-6}},
    "gradients": False,
    'center_atom_weight': 1.0,
}

In [ ]:
sys.path.append('/Users/rca/source_installs/alchemical-learning')
from utils.combine.alchemical import _species_coupling_matrix
M = _species_coupling_matrix(species)
pca = PCA()
M_PC = pca.fit_transform(M)
plt.semilogy(np.cumsum(pca.explained_variance_ratio_[:100]))
N_PSEUDO = np.where(np.cumsum(pca.explained_variance_ratio_[:100])>0.99)[0][0]
N_PSEUDO

In [ ]:
import numpy as np
import torch
import cProfile
import time
import copy

import matplotlib.pyplot as plt

import ase.io

import sklearn.model_selection

import sys
sys.path.append('/Users/rca/source_installs/alchemical-learning/')
from utils.dataset import AtomisticDataset, create_dataloader
from utils.soap import PowerSpectrum

from utils.combine import CombineSpecies, CombineRadial, CombineRadialSpecies
from utils.combine.alchemical import _species_coupling_matrix
from utils.linear import LinearModel
from utils.operations import SumStructures, remove_gradient
from equistore import TensorMap, Labels, TensorBlock


torch.set_default_dtype(torch.float64)
class CombinedLinearModel(torch.nn.Module):
    def __init__(self, 
        combiner,
    ):
        super().__init__()

        self.sum_structure = SumStructures()
        self.combiner = combiner
        self.power_spectrum = PowerSpectrum()

    def forward(self, dataloader):
        raw_power_spectra = []
        indices = []
        for (spherical_expansion, _, _), idx in zip(dataloader, tqdm(dataloader.batch_sampler)):
            combined = self.combiner(spherical_expansion)
            power_spectrum = self.power_spectrum(combined)
            for i, p in zip(idx, self.sum_structure(power_spectrum).block().values):
                raw_power_spectra.append(p.detach().numpy())
                indices.append(i)
        return np.array(raw_power_spectra)[np.argsort(indices)].reshape((len(indices), -1)), np.array(indices)[np.argsort(indices)]

In [ ]:
combiner = CombineSpecies(species=species, n_pseudo_species=N_PSEUDO)
clm = CombinedLinearModel(combiner)

In [ ]:
import pickle
fn = f'power_spectrum_trial_{N_PSEUDO}-species.npz'
ifn =  fn.replace('power_spectrum', 'indices')

if not os.path.exists(fn):

    if not os.path.exists(ifn):
        n_trial = 5000
        i_trial_3dcd = np.random.choice(len(frames_3dcd), size=n_trial//2)
        i_trial_mp = np.random.choice(len(frames_mp), size=n_trial//2)

        # removing the frames that have very close atoms
        for i,v in enumerate(i_trial_3dcd):
            dists = frames_3dcd[v].get_all_distances()
            np.fill_diagonal(dists, 100.0)
            while np.min(dists)<0.1:
                v = np.random.choice(len(frames_3dcd))
                if v not in i_trial_3dcd:
                    i_trial_3dcd[i] = v
                    dists = frames_3dcd[v].get_all_distances()
                    np.fill_diagonal(dists, 100.0)

        for i,v in enumerate(i_trial_mp):
            dists = frames_mp[v].get_all_distances()
            np.fill_diagonal(dists, 100.0)
            while np.min(dists)<0.1:
                v = np.random.choice(len(frames_mp))
                if v not in i_trial_mp:
                    i_trial_mp[i] = v
                    dists = frames_mp[v].get_all_distances()
                    np.fill_diagonal(dists, 100.0)
        np.savez(file=ifn, i_trial_3dcd=i_trial_3dcd, i_trial_mp=i_trial_mp)
    else:
        d = np.load(ifn)
        i_trial_3dcd = d['i_trial_3dcd']
        i_trial_mp = d['i_trial_mp']
                
    trial_frames = np.concatenate(([frames_3dcd[i] for i in i_trial_3dcd],
                                   [frames_mp[i] for i in i_trial_mp]))
    print(fn)
    temp_ds = AtomisticDataset(trial_frames, species, hypers, save_spherical_expansions=False, 
                               energies=torch.Tensor(np.zeros((len(trial_frames),1))))
    temp_dl = create_dataloader(temp_ds, 10, shuffle=False, device='cpu')
    ps = clm.forward(temp_dl)

    np.savez(file=fn, arr=ps[0], i_trial_3dcd=i_trial_3dcd, i_trial_mp=i_trial_mp)
    
    x_scaler = StandardFlexibleScaler(column_wise=False).fit(ps[0])
    x = x_scaler.transform(ps[0])
    del ps
    pickle.dump(x_scaler, open(f'x_scaler-{N_PSEUDO}-species.pkl', 'wb'))
else:
    f = np.load(fn)
    idx = np.load(ifn)
    for i, j in zip(idx['i_trial_mp'], f['i_trial_mp']):
        if i != j:
            raise AssertionError("Datasets do not match")
    for i, j in zip(idx['i_trial_3dcd'], f['i_trial_3dcd']):
        if i != j:
            raise AssertionError("Datasets do not match")
    x_scaler = pickle.load(open(f'x_scaler-{N_PSEUDO}-species.pkl', 'rb'))
    x = x_scaler.transform(f['arr'])
    i_trial_3dcd = f['i_trial_3dcd']
    i_trial_mp = f['i_trial_mp']

In [ ]:
pca = PCA(n_components=2000)
pca.fit(x)
plt.semilogy(pca.explained_variance_ratio_[:2000])

In [ ]:
fn = f'pca-{N_PSEUDO}-species.pkl'
if not os.path.exists(fn):
    pca = PCA(n_components=2000)
    pca.fit(x)
    plt.semilogy(pca.explained_variance_ratio_[:2000])
    pickle.dump(pca, open(fn, 'wb'))
else:
    pca = pickle.load(open(fn, 'rb'))

In [ ]:
t = pca.transform(x)
del x
plt.scatter(t[:,0], t[:,1])

In [ ]:
from chemiscope import show as cshow
trial_frames = np.concatenate(([frames_3dcd[i] for i in i_trial_3dcd],
                                   [frames_mp[i] for i in i_trial_mp]))
source = np.zeros(len(trial_frames))
source[:len(source)//2] = 1
widget = cshow(frames=trial_frames, properties={"pca": t[:, :10], "source":source})
widget.save('temp.json')

In [ ]:
pbar = tqdm(total=len(frames_3dcd))

In [ ]:
from IPython.display import clear_output
fn = f'power_spectrum_pca_3dcd_{N_PSEUDO}-species.npy'
if not os.path.exists(fn):
    ps_pca = np.nan * np.ones((len(frames_3dcd), pca.n_components_))
else:
    ps_pca = np.load(fn)
    
pbar.reset(total=len(frames_3dcd))
print(fn)
for arr in np.array_split(range(len(frames_3dcd)), 100):
    ps = None
    temp_ds=None
    temp_dl=None
    frame_subset = np.array(frames_3dcd, dtype=object)[arr]
    if np.isnan(ps_pca[arr]).any():
        clear_output()
        temp_ds = AtomisticDataset(frame_subset, species, hypers, save_spherical_expansions=False, 
                                   energies=torch.Tensor(np.zeros((len(frame_subset),1))))
        temp_dl = create_dataloader(temp_ds, 10, shuffle=False, device='cpu')
        ps = clm.forward(temp_dl)
        ps_pca[arr] = pca.transform(x_scaler.transform(ps[0]))

        np.save(file=fn, arr=ps_pca)
    pbar.update(len(frame_subset))
del frames_3dcd

In [ ]:
pbar = tqdm(total=len(frames_mp))

In [ ]:
from IPython.display import clear_output
fn = f'power_spectrum_pca_mp_{N_PSEUDO}-species.npy'
if not os.path.exists(fn):
    ps_pca = np.nan * np.ones((len(frames_mp), pca.n_components_))
else:
    ps_pca = np.load(fn)
    
print(fn)
for arr in np.array_split(range(len(frames_mp)), 100):
    ps = None
    if np.isnan(ps_pca[arr]).any():
        ps = None
        temp_ds=None
        temp_dl=None
        clear_output()
        frame_subset = np.array(frames_mp, dtype=object)[arr]
        temp_ds = AtomisticDataset(frame_subset, species, hypers, save_spherical_expansions=False, 
                                   energies=torch.Tensor(np.zeros((len(frame_subset),1))))
        temp_dl = create_dataloader(temp_ds, 10, shuffle=False, device='cpu')
        ps = clm.forward(temp_dl)
        ps_pca[arr] = pca.transform(x_scaler.transform(ps[0]))
        pbar.update(len(frame_subset))

        np.save(file=fn, arr=ps_pca)
del frames_mp